In [4]:
x=1

In [5]:
import anndata as ad
import numpy as np
import scanpy as sc
import json

# unpaired output

In [6]:
adata = sc.read('/lustre/groups/ml01/workspace/anastasia.litinetskaya/notebooks/snakemake/trimodal/data/glue_trimodal_unpaired_integrated_highly_var.h5ad')
adata

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/sc/lib/python3.10/site-packages/anndata/_core/anndata.py:117: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/sc/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


AnnData object with n_obs × n_vars = 147902 × 0
    obs: 'GEX_pct_counts_mt', 'GEX_size_factors', 'GEX_phase', 'cell_type', 'batch', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'new_batch', 'balancing_weight', 'assay'
    uns: 'Samplename_colors', 'assay_colors', 'cell_type_colors', 'neighbors', 'umap'
    obsm: 'GEX_X_pca', 'GEX_X_umap', 'X_glue', 'X_umap'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [4]:
with open('/lustre/groups/ml01/workspace/anastasia.litinetskaya/trimodal_suppl_benchmarks_data/trimodal/cellttype_harmonize.json', 'r') as f:
    harmonized_celltypes = json.load(f)
harmonized_celltypes.keys()

dict_keys(['cite_ct_l1_map', 'cite_ct_l2_map', 'multi_ct_l1_map', 'multi_ct_l2_map'])

In [5]:
adata.obs['Modality']

TAGTTGTCACCCTCAC-1-s1d1    multiome
CTATGGCCATAACGGG-1-s1d1    multiome
CCGCACACAGGTTAAA-1-s1d1    multiome
TCATTTGGTAATGGAA-1-s1d1    multiome
ACCACATAGGTGTCCA-1-s1d1    multiome
                             ...   
TCCGGGAGTTGGGTTT-1-s2d5        cite
TTGGATGAGGTGCCAA-1-s2d5        cite
ACAGAAACACAGCGCT-1-s2d5        cite
CACACAAGTCTAATCG-1-s2d5        cite
GTCCACTAGCTACGTT-1-s2d5        cite
Name: Modality, Length: 147902, dtype: category
Categories (2, object): ['cite', 'multiome']

In [6]:
adata.obs['cell_type'][adata.obs['Modality'].isin(['cite'])]

GCATTAGCATAAGCGG-1-s1d1    Naive CD20+ B IGKC+
TACAGGTGTTAGAGTA-1-s1d1             CD14+ Mono
AGGATCTAGGTCTACT-1-s1d1    Naive CD20+ B IGKC+
GTAGAAAGTGACACAG-1-s1d1                    HSC
TCCGAAAAGGATCATA-1-s1d1           Reticulocyte
                                  ...         
TCCGGGAGTTGGGTTT-1-s2d5                     NK
TTGGATGAGGTGCCAA-1-s2d5            NK CD158e1+
ACAGAAACACAGCGCT-1-s2d5    Naive CD20+ B IGKC+
CACACAAGTCTAATCG-1-s2d5           CD4+ T naive
GTCCACTAGCTACGTT-1-s2d5           CD4+ T naive
Name: cell_type, Length: 82964, dtype: category
Categories (48, object): ['B1 B', 'B1 B IGKC+', 'B1 B IGKC-', 'CD4+ T CD314+ CD45RA+', ..., 'dnT', 'gdT CD158b+', 'gdT TCRVD2+', 'pDC']

In [7]:
adata.obs['cell_type'][adata.obs['Modality'].isin(['multiome'])]

TAGTTGTCACCCTCAC-1-s1d1    Naive CD20+ B
CTATGGCCATAACGGG-1-s1d1       CD14+ Mono
CCGCACACAGGTTAAA-1-s1d1           CD8+ T
TCATTTGGTAATGGAA-1-s1d1           CD8+ T
ACCACATAGGTGTCCA-1-s1d1       CD16+ Mono
                               ...      
TCGTGCTTCGTTACAA-6-s2d5               NK
TGTGGAGCAGGTTAAA-6-s2d5       CD16+ Mono
TTCGCAACATGATTGT-6-s2d5           CD8+ T
AAGTAGCCATGTGGGA-6-s2d5               NK
CTCACTCAGCCTTAAA-6-s2d5       CD14+ Mono
Name: cell_type, Length: 64938, dtype: category
Categories (48, object): ['B1 B', 'B1 B IGKC+', 'B1 B IGKC-', 'CD4+ T CD314+ CD45RA+', ..., 'dnT', 'gdT CD158b+', 'gdT TCRVD2+', 'pDC']

In [8]:
adata.obs['l2_cell_type'] = 'test'
multiome_mask = adata.obs['Modality'] == 'multiome'
cite_mask = adata.obs['Modality'] == 'cite'
adata.obs.loc[multiome_mask, 'l2_cell_type'] = adata.obs['cell_type'].loc[multiome_mask].map(harmonized_celltypes['multi_ct_l2_map'])
adata.obs.loc[cite_mask, 'l2_cell_type'] = adata.obs['cell_type'].loc[cite_mask].map(harmonized_celltypes['cite_ct_l2_map'])

In [9]:
adata.obs.loc[multiome_mask, 'l2_cell_type']

TAGTTGTCACCCTCAC-1-s1d1       Naive CD20+ B
CTATGGCCATAACGGG-1-s1d1          CD14+ Mono
CCGCACACAGGTTAAA-1-s1d1    CD8+ T activated
TCATTTGGTAATGGAA-1-s1d1    CD8+ T activated
ACCACATAGGTGTCCA-1-s1d1          CD16+ Mono
                                 ...       
TCGTGCTTCGTTACAA-6-s2d5                  NK
TGTGGAGCAGGTTAAA-6-s2d5          CD16+ Mono
TTCGCAACATGATTGT-6-s2d5    CD8+ T activated
AAGTAGCCATGTGGGA-6-s2d5                  NK
CTCACTCAGCCTTAAA-6-s2d5          CD14+ Mono
Name: l2_cell_type, Length: 64938, dtype: object

In [10]:
adata.obs.loc[cite_mask, 'l2_cell_type']

GCATTAGCATAAGCGG-1-s1d1    Naive CD20+ B
TACAGGTGTTAGAGTA-1-s1d1       CD14+ Mono
AGGATCTAGGTCTACT-1-s1d1    Naive CD20+ B
GTAGAAAGTGACACAG-1-s1d1              HSC
TCCGAAAAGGATCATA-1-s1d1     Reticulocyte
                               ...      
TCCGGGAGTTGGGTTT-1-s2d5               NK
TTGGATGAGGTGCCAA-1-s2d5               NK
ACAGAAACACAGCGCT-1-s2d5    Naive CD20+ B
CACACAAGTCTAATCG-1-s2d5     CD4+ T naive
GTCCACTAGCTACGTT-1-s2d5     CD4+ T naive
Name: l2_cell_type, Length: 82964, dtype: object

In [11]:
adata

AnnData object with n_obs × n_vars = 147902 × 0
    obs: 'GEX_pct_counts_mt', 'GEX_size_factors', 'GEX_phase', 'cell_type', 'batch', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'new_batch', 'balancing_weight', 'assay', 'l2_cell_type'
    uns: 'Samplename_colors', 'assay_colors', 'cell_type_colors', 'neighbors', 'umap'
    obsm: 'GEX_X_pca', 'GEX_X_umap', 'X_glue', 'X_umap'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [12]:
adata.obsm['latent'] = adata.obsm['X_glue'].copy()
adata

AnnData object with n_obs × n_vars = 147902 × 0
    obs: 'GEX_pct_counts_mt', 'GEX_size_factors', 'GEX_phase', 'cell_type', 'batch', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'new_batch', 'balancing_weight', 'assay', 'l2_cell_type'
    uns: 'Samplename_colors', 'assay_colors', 'cell_type_colors', 'neighbors', 'umap'
    obsm: 'GEX_X_pca', 'GEX_X_umap', 'X_glue', 'X_umap', 'latent'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [13]:
adata.obsm['latent'].shape

(147902, 50)

In [14]:
adata.write('/lustre/groups/ml01/workspace/anastasia.litinetskaya/notebooks/snakemake/trimodal/data/glue_unpaired_trimodal_integrated_clean.h5ad')
adata

AnnData object with n_obs × n_vars = 147902 × 0
    obs: 'GEX_pct_counts_mt', 'GEX_size_factors', 'GEX_phase', 'cell_type', 'batch', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'new_batch', 'balancing_weight', 'assay', 'l2_cell_type'
    uns: 'Samplename_colors', 'assay_colors', 'cell_type_colors', 'neighbors', 'umap'
    obsm: 'GEX_X_pca', 'GEX_X_umap', 'X_glue', 'X_umap', 'latent'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [7]:
adata = sc.read('/lustre/groups/ml01/workspace/anastasia.litinetskaya/notebooks/snakemake/trimodal/data/glue_unpaired_trimodal_integrated_clean.h5ad')
adata

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/sc/lib/python3.10/site-packages/anndata/_core/anndata.py:117: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/sc/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


AnnData object with n_obs × n_vars = 147902 × 0
    obs: 'GEX_pct_counts_mt', 'GEX_size_factors', 'GEX_phase', 'cell_type', 'batch', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'new_batch', 'balancing_weight', 'assay', 'l2_cell_type'
    uns: 'Samplename_colors', 'assay_colors', 'cell_type_colors', 'neighbors', 'umap'
    obsm: 'GEX_X_pca', 'GEX_X_umap', 'X_glue', 'X_umap', 'latent'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [8]:
adata.obs['Modality'] = adata.obs['Modality'].astype(str) + '_' + adata.obs['assay'].astype(str)
adata.obs['Modality']

TAGTTGTCACCCTCAC-1-s1d1    multiome_rna
CTATGGCCATAACGGG-1-s1d1    multiome_rna
CCGCACACAGGTTAAA-1-s1d1    multiome_rna
TCATTTGGTAATGGAA-1-s1d1    multiome_rna
ACCACATAGGTGTCCA-1-s1d1    multiome_rna
                               ...     
TCCGGGAGTTGGGTTT-1-s2d5        cite_adt
TTGGATGAGGTGCCAA-1-s2d5        cite_adt
ACAGAAACACAGCGCT-1-s2d5        cite_adt
CACACAAGTCTAATCG-1-s2d5        cite_adt
GTCCACTAGCTACGTT-1-s2d5        cite_adt
Name: Modality, Length: 147902, dtype: object

In [9]:
adata.write('/lustre/groups/ml01/workspace/anastasia.litinetskaya/notebooks/snakemake/trimodal/data/glue_unpaired_trimodal_integrated_clean.h5ad')

# averaged output

In [15]:
adata = sc.read('/lustre/groups/ml01/workspace/anastasia.litinetskaya/notebooks/snakemake/trimodal/data/glue_unpaired_trimodal_integrated_clean.h5ad')
adata

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/sc/lib/python3.10/site-packages/anndata/_core/anndata.py:117: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/sc/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


AnnData object with n_obs × n_vars = 147902 × 0
    obs: 'GEX_pct_counts_mt', 'GEX_size_factors', 'GEX_phase', 'cell_type', 'batch', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'new_batch', 'balancing_weight', 'assay', 'l2_cell_type'
    uns: 'Samplename_colors', 'assay_colors', 'cell_type_colors', 'neighbors', 'umap'
    obsm: 'GEX_X_pca', 'GEX_X_umap', 'X_glue', 'X_umap', 'latent'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [16]:
adata.obs['Modality'].cat.categories

Index(['cite', 'multiome'], dtype='object')

In [17]:
adata.obs['assay'].cat.categories

Index(['adt', 'atac', 'rna'], dtype='object')

In [18]:
rna_multiome = adata[(adata.obs['Modality'] == 'multiome') & (adata.obs['assay'] == 'rna')].copy()
rna_multiome

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/sc/lib/python3.10/site-packages/anndata/_core/anndata.py:117: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


AnnData object with n_obs × n_vars = 32469 × 0
    obs: 'GEX_pct_counts_mt', 'GEX_size_factors', 'GEX_phase', 'cell_type', 'batch', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'new_batch', 'balancing_weight', 'assay', 'l2_cell_type'
    uns: 'Samplename_colors', 'assay_colors', 'cell_type_colors', 'neighbors', 'umap'
    obsm: 'GEX_X_pca', 'GEX_X_umap', 'X_glue', 'X_umap', 'latent'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [19]:
atac_multiome = adata[(adata.obs['Modality'] == 'multiome') & (adata.obs['assay'] == 'atac')].copy()
atac_multiome

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/sc/lib/python3.10/site-packages/anndata/_core/anndata.py:117: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


AnnData object with n_obs × n_vars = 32469 × 0
    obs: 'GEX_pct_counts_mt', 'GEX_size_factors', 'GEX_phase', 'cell_type', 'batch', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'new_batch', 'balancing_weight', 'assay', 'l2_cell_type'
    uns: 'Samplename_colors', 'assay_colors', 'cell_type_colors', 'neighbors', 'umap'
    obsm: 'GEX_X_pca', 'GEX_X_umap', 'X_glue', 'X_umap', 'latent'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [20]:
np.sum(rna_multiome.obs_names != atac_multiome.obs_names)

0

In [21]:
rna_cite = adata[(adata.obs['Modality'] == 'cite') & (adata.obs['assay'] == 'rna')].copy()
rna_cite

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/sc/lib/python3.10/site-packages/anndata/_core/anndata.py:117: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


AnnData object with n_obs × n_vars = 41482 × 0
    obs: 'GEX_pct_counts_mt', 'GEX_size_factors', 'GEX_phase', 'cell_type', 'batch', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'new_batch', 'balancing_weight', 'assay', 'l2_cell_type'
    uns: 'Samplename_colors', 'assay_colors', 'cell_type_colors', 'neighbors', 'umap'
    obsm: 'GEX_X_pca', 'GEX_X_umap', 'X_glue', 'X_umap', 'latent'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [22]:
adt_cite = adata[(adata.obs['Modality'] == 'cite') & (adata.obs['assay'] == 'adt')].copy()
adt_cite

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/sc/lib/python3.10/site-packages/anndata/_core/anndata.py:117: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


AnnData object with n_obs × n_vars = 41482 × 0
    obs: 'GEX_pct_counts_mt', 'GEX_size_factors', 'GEX_phase', 'cell_type', 'batch', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'new_batch', 'balancing_weight', 'assay', 'l2_cell_type'
    uns: 'Samplename_colors', 'assay_colors', 'cell_type_colors', 'neighbors', 'umap'
    obsm: 'GEX_X_pca', 'GEX_X_umap', 'X_glue', 'X_umap', 'latent'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [23]:
np.sum(rna_cite.obs_names != adt_cite.obs_names)

0

In [24]:
multiome = sc.AnnData(X = 0.5*(rna_multiome.obsm['latent'] + atac_multiome.obsm['latent']), obs=rna_multiome.obs)
multiome

AnnData object with n_obs × n_vars = 32469 × 50
    obs: 'GEX_pct_counts_mt', 'GEX_size_factors', 'GEX_phase', 'cell_type', 'batch', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'new_batch', 'balancing_weight', 'assay', 'l2_cell_type'

In [25]:
multiome.obsm['latent'] = multiome.X.copy()

In [26]:
cite = sc.AnnData(X = 0.5*(rna_cite.obsm['latent'] + adt_cite.obsm['latent']), obs=rna_cite.obs)
cite

AnnData object with n_obs × n_vars = 41482 × 50
    obs: 'GEX_pct_counts_mt', 'GEX_size_factors', 'GEX_phase', 'cell_type', 'batch', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'new_batch', 'balancing_weight', 'assay', 'l2_cell_type'

In [27]:
cite.obsm['latent'] = cite.X.copy()

In [28]:
latent = ad.concat([multiome, cite])
latent

AnnData object with n_obs × n_vars = 73951 × 50
    obs: 'GEX_pct_counts_mt', 'GEX_size_factors', 'GEX_phase', 'cell_type', 'batch', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'new_batch', 'balancing_weight', 'assay', 'l2_cell_type'
    obsm: 'latent'

In [29]:
latent.write('/lustre/groups/ml01/workspace/anastasia.litinetskaya/notebooks/snakemake/trimodal/data/averaged_glue_unpaired_trimodal_integrated_clean.h5ad')